In [1]:
display(dbutils.fs.ls("/mnt/s3data"))

path,name,size
dbfs:/mnt/s3data/Superstore.csv,Superstore.csv,2287742
dbfs:/mnt/s3data/returns.csv,returns.csv,16019


In [2]:
# File location and type
file_location = "dbfs:/mnt/s3data/Superstore.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_superstore = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

#display(df_superstore.printSchema())

In [3]:
file_location = "dbfs:/mnt/s3data/returns.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_returns = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location) \
  .dropDuplicates()


In [4]:
# join df_superstore and df_returns
df_superstore_merge=df_superstore.join(df_returns,"Order ID",how="left")
display(df_superstore_merge)

Order ID,Row ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Returned
CA-2018-152156,1,11/8/2018,11/11/2018,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0,41.9136,null
CA-2018-152156,2,11/8/2018,11/11/2018,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0,219.582,null
CA-2018-138688,3,6/12/2018,6/16/2018,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0,6.8714,null
US-2017-108966,4,10/11/2017,10/18/2017,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.031,null
US-2017-108966,5,10/11/2017,10/18/2017,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.368,2,0.2,2.5164,null
CA-2016-115812,6,6/9/2016,6/14/2016,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,FUR-FU-10001487,Furniture,Furnishings,"Eldon Expressions Wood and Plastic Desk Accessories, Cherry Wood",48.86,7,0,14.1694,null
CA-2016-115812,7,6/9/2016,6/14/2016,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,OFF-AR-10002833,Office Supplies,Art,Newell 322,7.28,4,0,1.9656,null
CA-2016-115812,8,6/9/2016,6/14/2016,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,TEC-PH-10002275,Technology,Phones,Mitel 5320 IP Phone VoIP phone,907.152,6,0.2,90.7152,null
CA-2016-115812,9,6/9/2016,6/14/2016,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,OFF-BI-10003910,Office Supplies,Binders,DXL Angle-View Binders with Locking Rings by Samsill,18.504,3,0.2,5.7825,null
CA-2016-115812,10,6/9/2016,6/14/2016,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,OFF-AP-10002892,Office Supplies,Appliances,Belkin F5C206VTEL 6 Outlet Surge,114.9,5,0,34.47,null


In [5]:
#Clean Column header remove spaces and special character
from pyspark.sql.types import DateType,IntegerType
from pyspark.sql.functions import *
exprs = [col(column).alias(column.replace(' ', '_')) for column in df_superstore_merge.columns]
print(exprs)
df_superstore_cln=df_superstore_merge.select(*exprs)
df_superstore_cln=df_superstore_cln.withColumnRenamed("Sub-Category", "Sub_Category")\
       .withColumnRenamed("Country/Region", "Country_Region") \
       .filter(col("Country_Region")=="United States")
#display(df_superstore_cln)

[Column<b'Order ID AS `Order_ID`'>, Column<b'Row ID AS `Row_ID`'>, Column<b'Order Date AS `Order_Date`'>, Column<b'Ship Date AS `Ship_Date`'>, Column<b'Ship Mode AS `Ship_Mode`'>, Column<b'Customer ID AS `Customer_ID`'>, Column<b'Customer Name AS `Customer_Name`'>, Column<b'Segment AS `Segment`'>, Column<b'Country/Region AS `Country/Region`'>, Column<b'City AS `City`'>, Column<b'State AS `State`'>, Column<b'Postal Code AS `Postal_Code`'>, Column<b'Region AS `Region`'>, Column<b'Product ID AS `Product_ID`'>, Column<b'Category AS `Category`'>, Column<b'Sub-Category AS `Sub-Category`'>, Column<b'Product Name AS `Product_Name`'>, Column<b'Sales AS `Sales`'>, Column<b'Quantity AS `Quantity`'>, Column<b'Discount AS `Discount`'>, Column<b'Profit AS `Profit`'>, Column<b'Returned AS `Returned`'>]

In [6]:
#Convert Date column and calculate duration
df_superstore_cln=df_superstore_cln.withColumn("OrderDateClean",to_date(col("Order_Date"), "MM/dd/yyyy")).withColumn("ShipDateClean",to_date(col("Ship_Date"), "MM/dd/yyyy")).withColumn("duration",datediff(col("ShipDateClean"),col("OrderDateClean")))
#display(df_superstore_cln)

In [7]:
# Aggegate and calcuate Sales, Qty, Avg Duration and Uniqe Customer by state, Category, sub cat and ship mode grain
df_superstore_sales=df_superstore_cln.groupby("State","Category","Sub_Category","Ship_Mode").agg({'Sales':'sum','Quantity':'sum','duration':'avg','Order_ID':'count'})
df_uniq_customer=df_superstore_cln.groupby("State","Category","Sub_Category","Ship_Mode").agg(countDistinct("Customer_ID"))
#df_superstore_agg=df_superstore_sales.join(df_uniq_customer,(df_superstore_sales.State == df_uniq_customer.State) & (df_superstore_sales.Category == df_uniq_customer.Category) & (df_superstore_sales.Sub_Category == df_uniq_customer.Sub_Category) & (df_superstore_sales.Ship_Mode == df_uniq_customer.Ship_Mode))
df_superstore_agg=df_superstore_sales.join(df_uniq_customer,["State","Category","Sub_Category","Ship_Mode"])
display(df_superstore_agg)

State,Category,Sub_Category,Ship_Mode,avg(duration),count(Order_ID),sum(Sales),sum(Quantity),count(DISTINCT Customer_ID)
Colorado,Office Supplies,Envelopes,Standard Class,6.0,1,12.536,1.0,1
Oklahoma,Office Supplies,Paper,Standard Class,5.833333333333333,6,183.66,16.0,6
Oklahoma,Office Supplies,Storage,Standard Class,5.75,4,1470.72,550.24,4
Oregon,Furniture,Chairs,Second Class,2.0,2,755.136,4.0,1
Pennsylvania,Office Supplies,Envelopes,Standard Class,4.466666666666667,15,675.736,50.0,15
South Dakota,Office Supplies,Binders,Standard Class,5.0,2,103.83,12.0,1
Arizona,Furniture,Bookcases,First Class,1.5,2,391.449,12.0,2
Arizona,Furniture,Tables,First Class,2.0,1,393.165,3.0,1
Arizona,Furniture,Tables,Standard Class,6.0,7,3419.6349999999998,31.0,7
California,Furniture,Tables,Second Class,2.933333333333333,15,10202.48,60.0,14


In [8]:
# Prepare Final Dataframe(rename column name)
df_superstore_final=df_superstore_agg.select("State","Category","Sub_Category","Ship_Mode",round(col("avg(duration)"),2).alias("Avg_Duration"),col("count(Order_ID)").alias("UniqUserCountLong"),round(col("sum(Sales)"),2).alias("SalesAmt"),round(col("sum(Quantity)"),2).alias("Quantity"))

df_superstore_final=df_superstore_final.withColumn("UniqUserCount", df_superstore_final["UniqUserCountLong"].cast(IntegerType())).drop("UniqUserCountLong")
df_superstore_final.printSchema()
#display(df_superstore_final)


root
-- State: string (nullable = true)
-- Category: string (nullable = true)
-- Sub_Category: string (nullable = true)
-- Ship_Mode: string (nullable = true)
-- Avg_Duration: double (nullable = true)
-- SalesAmt: double (nullable = true)
-- Quantity: double (nullable = true)
-- UniqUserCount: integer (nullable = false)

In [9]:
# Load into a Table
spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")

superstore_table_name = "superstore_curated"

df_superstore_final.write.format("parquet").mode('overwrite').saveAsTable(superstore_table_name)